# 核心功能与使用示例

> 读取因子数据的核心模块

In [ ]:
#| default_exp __init__

In [ ]:
#| export
import requests
import pandas as pd
from typing import Iterable,Union
from functools import wraps,reduce
import json
import datetime
from tenacity import retry, stop_after_attempt,wait_fixed
import tqdm

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.config import get_config

In [ ]:
#| export
try:
    response = requests.get("https://pypi.org/pypi/factor-reader/json", timeout=2)
    latest_version = response.json()["info"]["version"]
    now_version=get_config(cfg_name='settings.ini')['version']
    if latest_version!=now_version:
        print(f'''您当前使用的是{now_version}，最新版本为{latest_version}
              建议您使用`pip install factor_reader --upgrade`
              或`pip install -i https://pypi.tuna.tsinghua.edu.cn/simple factor_reader --upgrade`命令进行更新
              ''')
except Exception:
    try:
        response = requests.get("https://pypi.org/pypi/factor-reader/json", timeout=2)
        latest_version = response.json()["info"]["version"]
        now_version=__version__
        if latest_version!=now_version:
            print(f'''您当前使用的是{now_version}，最新版本为{latest_version}
              建议您使用`pip install factor_reader --upgrade`
              或`pip install -i https://pypi.tuna.tsinghua.edu.cn/simple factor_reader --upgrade`命令进行更新
              ''')
    except Exception:
        ...

In [ ]:
# | export
class TokenUnavailableError(PermissionError):
    def __init__(self,error_info):
        super().__init__(self) 
        self.error_info=error_info
        
    def __str__(self):
        return self.error_info


def do_on_dfs_class(func):
    """一个装饰器，使类的方法的参数可以输入为列表或元组，返回值为分别对各个参数执行此函数后依次得到的结果"""

    @wraps(func)
    def wrapper(self, df=None, *args, **kwargs):
        if isinstance(df, list) or isinstance(df, tuple):
            dfs = [func(self, i, *args, **kwargs) for num, i in enumerate(df)]
            return dfs
        else:
            return func(self, df, *args, **kwargs)

    return wrapper

def concat_nosqls(func):
    @wraps(func)
    def wrapper(self,fac_key,*args,**kwargs):
        dfs=func(self,fac_key,*args,**kwargs)
        def double_index(res,key):
            res=res.reset_index().assign(name=self._deal_key_name(key)[1])
            res=res.set_index(['name',list(res.columns)[0]])
            return res
        if (isinstance(dfs, list) or isinstance(dfs, tuple)) and ("date" not in dfs[0].columns):
            dfs=[double_index(i,self._deal_key_name(key)[1]) for i,key in zip(dfs,fac_key)]
            dfs=pd.concat(dfs)
        elif (isinstance(dfs, list) or isinstance(dfs, tuple)) and ("date" in dfs[0].columns):
            dfs = reduce(
                lambda x, y: pd.merge(x, y, on=["date", "code"], how="outer"), dfs
            )
        elif "date" not in dfs.columns:
            dfs=double_index(dfs,fac_key)
        return dfs
    return wrapper



In [ ]:
# | export
class FactorReader():
    def __init__(self,token:str) -> None:
        self.token=token
        self.host='http://43.143.223.158:1838/'
        infos=self.show_all_factors_information()
        self.keys=list(infos.数据键名)
        self.names=list(infos.因子名称)
        self.keys_names = {k: v for k, v in zip(self.keys, self.names)}
        self.names_keys = {k: v for k, v in zip(self.names, self.keys)}
        
    def _deal_key_name(self,input_key):
        names_related = [i for i in self.names if input_key in i]
        if input_key in self.keys:
            fac_key=input_key
            fac_name=self.keys_names[fac_key]
        elif len(names_related) > 0:
            fac_name = names_related[0]
            fac_key = self.names_keys[fac_name]
        else:
            raise ValueError(
                "输入的fac_key参数有误，请输入因子名称或因子键名，可通过`factor_reader.show_all_factors_information`函数来查看可用的因子名称和键名"
            )
        return fac_key,fac_name
    
    @retry(stop=stop_after_attempt(100),wait=wait_fixed(3))
    def show_all_factors_information(self) -> pd.DataFrame:
        url=self.host+self.token+'/info'
        df=requests.get(url).text
        if df=='您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~':
            raise TokenUnavailableError('您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~')
        df=pd.read_json(json.loads(df))
        return df
    
    @concat_nosqls
    @do_on_dfs_class
    def read_factor(
        self,
        fac_key: str,
        trade_date: Union[int,str,datetime.datetime] = None,
        start_date: Union[int,str,datetime.datetime] = None,
        end_date: Union[int,str,datetime.datetime] = None,
        sql_like: bool = False,
    ) -> pd.DataFrame:
        """通过表名，读取因子数据

        Parameters
        ----------
        fac_key : str
            表的名称或因子的名称
        trade_date : Union[int,str,datetime.datetime], optional
            读取单日因子值，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13')，指定此参数时，start_date和end_date两个参数将失效, by default None
        start_date : Union[int,str,datetime.datetime], optional
            读取因子值的起始日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None
        end_date : Union[int,str,datetime.datetime], optional
            读取因子值的终止日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None
        sql_like : bool, optional
            返回的数据为形如sql中的长表，包括日期、股票代码、因子值三列, by default False

        Returns
        -------
        pd.DataFrame
            因子值，index为每天的日期，columns为股票代码，values为因子值
        """
        def date_ok(x):
            if isinstance(x,int):
                date=pd.Timestamp(str(x))
            elif isinstance(x,float):
                date=pd.Timestamp(str(int(x)))
            else:
                date=pd.Timestamp(x)
            date=datetime.datetime.strftime(date,'%Y%m%d')
            return date
        
        '''名称部分处理'''
        fac_key,fac_name=self._deal_key_name(fac_key)
        print(f"正在读取{fac_name}的数据")
        
        '''日期部分处理'''
        if trade_date is not None:
            trade_date=date_ok(trade_date)
            start_date='None'
            end_date='None'
            cuts=None
        elif (start_date is not None) and (end_date is not None):
            start_date=date_ok(start_date)
            end_date=date_ok(end_date)
            dura=pd.Timestamp(end_date)-pd.Timestamp(start_date)
            if dura>pd.Timedelta(days=30):
                cuts=pd.date_range(pd.Timestamp(start_date),pd.Timestamp(end_date))
                cuts=list(pd.Series([start_date]+[datetime.datetime.strftime(i,'%Y%m%d') for i in list(cuts)]+[end_date]).sort_values().drop_duplicates())
            else:
                cuts=None
            trade_date='None'
        elif start_date is not None:
            start_date=date_ok(start_date)
            trade_date='None'
            end_date='None'
            if dura>pd.Timedelta(days=30):
                cuts=pd.date_range(pd.Timestamp(start_date),datetime.datetime.now())
                cuts=list(pd.Series([start_date]+[datetime.datetime.strftime(i,'%Y%m%d') for i in list(cuts)+[datetime.datetime.now()]]).sort_values().drop_duplicates())
            else:
                cuts=None
        elif end_date is not None:
            end_date=date_ok(end_date)
            trade_date='None'
            start_date='None'
            cuts=None
        else:
            raise ValueError('请至少指定trade_date、start_date、end_date参数中的一个')
        
        @retry(stop=stop_after_attempt(100),wait=wait_fixed(3))
        def read_in(trade_date):
            url=self.host+self.token+'/'+fac_key+'/'+trade_date+'/'+start_date+'/'+end_date+'/'+str(int(sql_like))
            res=requests.get(url).text
            self.res=res
            if res=='您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~':
                raise TokenUnavailableError('您的token暂不可用，请检查核验后重试。或联系方正金工团队开通可用token~')
            res=pd.read_json(json.loads(res))
            if sql_like:
                res=res.stack().reset_index()
                res.columns=['date','code','fac']
                res=res.rename(columns={'fac':self.keys_names[fac_key]})
            else:
                ...
            return res
        
        if cuts is None:
            return read_in(trade_date=trade_date)
        else:
            print('您读取的数据过长，正在分段读取，请稍候……')
            ress=[]
            for trade_in in tqdm.tqdm(cuts):
                ress.append(read_in(trade_in))
            print('读取完成，正在拼接，请稍等')
            res=pd.concat(ress)
            return res

In [ ]:
show_doc(FactorReader.show_all_factors_information)

---

### FactorReader.show_all_factors_information

>      FactorReader.show_all_factors_information ()

In [ ]:
show_doc(FactorReader.read_factor)

---

### FactorReader.read_factor

>      FactorReader.read_factor (fac_key:str,
>                                trade_date:Union[int,str,datetime.datetime]=Non
>                                e, start_date:Union[int,str,datetime.datetime]=
>                                None,
>                                end_date:Union[int,str,datetime.datetime]=None,
>                                sql_like:bool=False)

通过表名，读取因子数据

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fac_key | str |  | 表的名称或因子的名称 |
| trade_date | Union | None | 读取单日因子值，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13')，指定此参数时，start_date和end_date两个参数将失效, by default None |
| start_date | Union | None | 读取因子值的起始日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None |
| end_date | Union | None | 读取因子值的终止日期，形如20230106或'20230106'或'2023-01-13'或pd.Timestamp('2023-01-13'), by default None |
| sql_like | bool | False | 返回的数据为形如sql中的长表，包括日期、股票代码、因子值三列, by default False |
| **Returns** | **DataFrame** |  | **因子值，index为每天的日期，columns为股票代码，values为因子值** |

# 使用示例

In [ ]:
token='pypi' #|hide_line

## 第一步：实例化一个FactorReader对象，输入token

In [ ]:
fr=FactorReader(token=token) # 初始化一个FactorReader对象，输入token，连接因子数据库

## 第二步：查看当前因子数据库中有哪些因子（此步骤可选）

In [34]:
fr.show_all_factors_information() # 展示所有可读取因子的相关信息

,数据键名,因子名称,报告题目,微信链接
0,factor1,适度冒险因子,成交量激增时刻蕴含的alpha信息——多因子选股系列研究之一,https://mp.weixin.qq.com/s/2pObrtp3V0dv50MFGg_fhw
1,factor2,完整潮汐因子,个股成交量的潮汐变化及“潮汐”因子构建——多因子选股系列研究之二,https://mp.weixin.qq.com/s/_2xWXM8iyNzYDYolT9vVsw
2,factor3,勇攀高峰因子,个股波动率的变动及”勇攀高峰“因子构建——多因子选股系列研究之三,https://mp.weixin.qq.com/s/-IxH3n-uR0BwOIbOyzqPrw
3,factor4,球队硬币因子,个股动量效应的识别及“球队硬币”因子构建——多因子选股系列研究之四,https://mp.weixin.qq.com/s/JCv8qziZ_ZfTbBE6l8cKOQ
4,factor5,云开雾散因子,波动率的波动率与投资者模糊性厌恶——多因子选股系列研究之五,https://mp.weixin.qq.com/s/vX4I9SpKRF3_HKQOGhtZcQ
5,factor6,飞蛾扑火因子,个股股价跳跃及其对振幅因子的改进——多因子选股系列研究之六,https://mp.weixin.qq.com/s/V8r1Xbz5J0A9D5J_9GD_IA
6,factor8,草木皆兵因子,显著效应、极端收益扭曲决策权重和“草木皆兵”因子——多因子选股系列研究之八,https://mp.weixin.qq.com/s/hdtzQaCF2h6ZjZd-0HAftQ
7,factor9,水中行舟因子,个股成交额的市场跟随性与“水中行舟”因子——多因子选股系列研究之九,https://mp.weixin.qq.com/s/Zv6XX8ddUvBCtG-4LM1MKQ


## 第三步：读取因子数据

### ①读取单日单个因子的数据

In [35]:
fr.read_factor('球队硬币',trade_date=20230106) # 读取球队硬币因子在2023年1月13日的因子数据

正在读取球队硬币因子的数据


,,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,871970.BJ,871981.BJ,872374.BJ,872808.BJ,872925.BJ,873122.BJ,873169.BJ,873223.BJ,873339.BJ,873527.BJ
name,index,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,0.104238,-1.345124,-1.070172,-3.192394,-1.628848,0.60037,-2.126244,-0.90467,-0.537109,-1.108383


In [36]:
fr.read_factor('factor4',trade_date='20230106') # 读取方正金工多因子系列第4篇的因子在2023年1月13日的因子数据

正在读取球队硬币因子的数据


,,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,871970.BJ,871981.BJ,872374.BJ,872808.BJ,872925.BJ,873122.BJ,873169.BJ,873223.BJ,873339.BJ,873527.BJ
name,index,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,0.104238,-1.345124,-1.070172,-3.192394,-1.628848,0.60037,-2.126244,-0.90467,-0.537109,-1.108383


### ②读取多个交易日单个因子的数据

In [ ]:
fr.read_factor('球队硬币',start_date='2023-01-01',end_date=pd.Timestamp('20230106')) # 读取球队硬币因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据


000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
name   index                                                               
球队硬币因子 2023-01-03  -2.358642  -1.345033   1.162492   0.007223   8.697314   
       2023-01-04  -3.250196  -0.612765   1.104373  -0.604997   8.853588   
       2023-01-05  -3.612259  -0.353330   0.683165  -0.712644   8.803313   
       2023-01-06  -3.458540  -0.674148   0.220243  -0.950252   8.786346   

                   000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  ...  \
name   index                                                              ...   
球队硬币因子 2023-01-03   1.189680  -1.702174  -1.052431  -0.854015   3.398331  ...   
       2023-01-04   0.434524  -2.027665  -1.078369  -1.319052   2.476316  ...   
       2023-01-05   0.304470  -1.978654  -0.557577  -1.869844   2.610491  ...   
       2023-01-06   0.500339  -1.828122   0.267091  -0.781898   2.299623  ...   

                   871970.BJ  871981.BJ  872374.BJ  872808.BJ  872925.BJ  \
name   index                                                               
球队硬币因子 2023-01-03  -0.103756  -1.137786  -1.021316  -3.253337  -1.775420   
       2023-01-04  -0.088794  -1.116035  -1.024373  -3.126951  -1.740175   
       2023-01-05   0.010475  -1.115315  -1.121056  -3.268384  -1.753120   
       2023-01-06   0.104238  -1.345124  -1.070172  -3.192394  -1.628848   

                   873122.BJ  873169.BJ  873223.BJ  873339.BJ  873527.BJ  
name   index                                                              
球队硬币因子 2023-01-03   2.468078  -1.900470  -1.004885  -0.407954  -0.799920  
       2023-01-04   2.741972  -2.164878  -0.959739  -0.397286  -0.845371  
       2023-01-05   1.020633  -2.132997  -0.893447  -0.449713  -0.817057  
       2023-01-06   0.600370  -2.126244  -0.904670  -0.537109  -1.108383  

[4 rows x 5017 columns]

In [ ]:
fr.read_factor('factor4',start_date=20230101,end_date=20230106,sql_like=True) # 以sql的形式，读取方正金工多因子系列第4篇的因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据


,date,code,球队硬币因子
0,2023-01-03,000001.SZ,-2.358642
1,2023-01-03,000002.SZ,-1.345033
2,2023-01-03,000004.SZ,1.162492
3,2023-01-03,000005.SZ,0.007223
4,2023-01-03,000006.SZ,8.697314
...,...,...,...
20045,2023-01-06,873122.BJ,0.600370
20046,2023-01-06,873169.BJ,-2.126244
20047,2023-01-06,873223.BJ,-0.904670
20048,2023-01-06,873339.BJ,-0.537109


### ③读取多个交易日多个因子的因子数据

In [ ]:
fr.read_factor(['球队硬币','云开雾散'],start_date='20230101',end_date='20230106') # 分别读取球队硬币因子和云开雾散因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据


000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
name   index                                                               
球队硬币因子 2023-01-03  -2.358642  -1.345033   1.162492   0.007223   8.697314   
       2023-01-04  -3.250196  -0.612765   1.104373  -0.604997   8.853588   
       2023-01-05  -3.612259  -0.353330   0.683165  -0.712644   8.803313   
       2023-01-06  -3.458540  -0.674148   0.220243  -0.950252   8.786346   
云开雾散因子 2023-01-03  -0.981326  -0.944349  -0.997003   4.198288   3.824620   
       2023-01-04  -0.874760  -0.565374  -1.418436  -2.256764   3.836005   
       2023-01-05  -0.254081  -0.614854  -1.407234  -2.364404   3.869275   
       2023-01-06  -0.291150  -0.524840  -1.476704  -3.120565   3.906959   

                   000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  ...  \
name   index                                                              ...   
球队硬币因子 2023-01-03   1.189680  -1.702174  -1.052431  -0.854015   3.398331  ...   
       2023-01-04   0.434524  -2.027665  -1.078369  -1.319052   2.476316  ...   
       2023-01-05   0.304470  -1.978654  -0.557577  -1.869844   2.610491  ...   
       2023-01-06   0.500339  -1.828122   0.267091  -0.781898   2.299623  ...   
云开雾散因子 2023-01-03  -0.305261  -2.440184   1.778184  -1.547923   2.822911  ...   
       2023-01-04  -0.601352  -1.189656   1.456942  -1.902164   2.427931  ...   
       2023-01-05  -1.362426  -1.225968   1.424493  -1.910795   2.395279  ...   
       2023-01-06  -1.295554  -1.253691  -0.280816   0.352601   2.508948  ...   

                   301398.SZ  600038.SH  600459.SH  600663.SH  600729.SH  \
name   index                                                               
球队硬币因子 2023-01-03        NaN        NaN        NaN        NaN        NaN   
       2023-01-04        NaN        NaN        NaN        NaN        NaN   
       2023-01-05        NaN        NaN        NaN        NaN        NaN   
       2023-01-06        NaN        NaN        NaN        NaN        NaN   
云开雾散因子 2023-01-03   2.320428  -0.672918  -0.150167   1.191432   0.075872   
       2023-01-04   2.093086  -0.477727  -0.157869   1.731492  -0.314115   
       2023-01-05   2.492977  -0.415604  -0.156190   1.573648   0.032488   
       2023-01-06   2.564407  -0.250353   0.183853   1.403672  -0.088864   

                   603029.SH  688141.SH  688147.SH  688172.SH  688498.SH  
name   index                                                              
球队硬币因子 2023-01-03        NaN        NaN        NaN        NaN        NaN  
       2023-01-04        NaN        NaN        NaN        NaN        NaN  
       2023-01-05        NaN        NaN        NaN        NaN        NaN  
       2023-01-06        NaN        NaN        NaN        NaN        NaN  
云开雾散因子 2023-01-03  10.216414        NaN        NaN   1.769576        NaN  
       2023-01-04  10.224871        NaN        NaN   1.559961   1.156921  
       2023-01-05   9.917723        NaN        NaN   1.374394   1.120350  
       2023-01-06  10.332152  -0.121816   0.644811   1.282682   0.950078  

[8 rows x 5033 columns]

In [ ]:
fr.read_factor(['球队硬币','云开雾散'],start_date='20230101',end_date='20230206')

正在读取球队硬币因子的数据
您读取的数据过长，正在分段读取，请稍候……


100%|██████████| 37/37 [00:28<00:00,  1.28it/s]


读取完成，正在拼接，请稍等
正在读取云开雾散因子的数据
您读取的数据过长，正在分段读取，请稍候……


100%|██████████| 37/37 [00:29<00:00,  1.27it/s]


读取完成，正在拼接，请稍等


000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
name   index                                                               
球队硬币因子 2023-01-03  -2.358642  -1.345033   1.162492   0.007223   8.697314   
       2023-01-04  -3.250196  -0.612765   1.104373  -0.604997   8.853588   
       2023-01-05  -3.612259  -0.353330   0.683165  -0.712644   8.803313   
       2023-01-06  -3.458540  -0.674148   0.220243  -0.950252   8.786346   
       2023-01-09  -3.354882  -0.170424   0.087265  -1.711030   8.091143   
       2023-01-10  -4.085837  -1.136618   0.041872  -1.484979   8.123063   
       2023-01-11  -4.299566  -1.225560   0.273944  -1.030241   7.683715   
       2023-01-12  -4.378207  -1.189928   0.237218  -0.655933   7.894500   
       2023-01-13  -4.479527  -1.476970   0.172361  -0.525152   7.266575   
       2023-01-16  -4.171016  -1.219680  -1.174946  -0.475743   6.070169   
       2023-01-17  -4.023675  -1.386111  -2.004400  -0.622768   5.288382   
       2023-01-18  -4.353316  -2.071842  -2.029811  -0.676816   5.694441   
       2023-01-19  -3.834700  -2.081139  -2.126112  -0.692192   5.532381   
       2023-01-20  -3.529300  -2.139061  -1.950409  -0.787610   6.188294   
       2023-01-30  -3.396520  -2.177579  -2.041623  -1.006709   4.699233   
       2023-01-31  -3.307280  -2.372615  -1.825028  -1.052096   4.660585   
       2023-02-01  -2.712186  -2.285143  -2.908093  -1.144443   2.583668   
       2023-02-02  -2.475493  -2.352708  -3.016441  -1.441300   1.523398   
       2023-02-03  -2.276289  -2.365098  -2.835789  -1.496792   3.269992   
       2023-02-06  -2.457774  -2.504629  -2.637363  -1.612015   3.456960   
云开雾散因子 2023-01-03  -0.981326  -0.944349  -0.997003   4.198288   3.824620   
       2023-01-04  -0.874760  -0.565374  -1.418436  -2.256764   3.836005   
       2023-01-05  -0.254081  -0.614854  -1.407234  -2.364404   3.869275   
       2023-01-06  -0.291150  -0.524840  -1.476704  -3.120565   3.906959   
       2023-01-09  -0.311834  -0.652490  -1.435259  -3.156991   3.906213   
       2023-01-10  -0.252993  -0.673770  -1.452988  -3.094026   4.145813   
       2023-01-11  -0.108614  -0.409360  -1.538049  -3.050843   4.022810   
       2023-01-12  -0.019939  -0.348093  -1.517629  -3.235680   4.079903   
       2023-01-13   0.004401  -0.231196  -1.393393  -3.245165   3.799570   
       2023-01-16  -0.095033  -0.481757  -1.352985  -3.607232   3.402758   
       2023-01-17   0.024778  -0.312762  -1.256656  -3.683347   3.316168   
       2023-01-18  -0.026242  -0.403705  -0.844195  -4.013815   3.142178   
       2023-01-19  -0.089630  -0.456148  -0.838379  -4.044224   3.142638   
       2023-01-20  -0.122600  -0.368921  -0.858872  -4.358661   2.865788   
       2023-01-30  -0.161589  -0.119212  -0.485579  -4.350704   2.349619   
       2023-01-31  -0.249149  -0.111299   0.580837  -4.311461   2.125479   
       2023-02-01  -0.244308  -0.094077   0.989772  -4.181594   1.518413   
       2023-02-02  -0.149381  -0.069716   0.750863  -4.240031   1.322263   
       2023-02-03  -0.324321  -0.017821   0.742932  -4.099188   1.124398   
       2023-02-06  -0.267734   0.007991   0.769199  -4.068778   0.035837   

                   000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  ...  \
name   index                                                              ...   
球队硬币因子 2023-01-03   1.189680  -1.702174  -1.052431  -0.854015   3.398331  ...   
       2023-01-04   0.434524  -2.027665  -1.078369  -1.319052   2.476316  ...   
       2023-01-05   0.304470  -1.978654  -0.557577  -1.869844   2.610491  ...   
       2023-01-06   0.500339  -1.828122   0.267091  -0.781898   2.299623  ...   
       2023-01-09   0.268643  -1.913154  -0.324667  -1.089943   1.179651  ...   
       2023-01-10   0.195857  -1.755869  -0.714262  -1.289059   1.667330  ...   
       2023-01-11   0.762255  -1.444859  -1.031784  -1.868811   1.372701  ...   
       2023-01-12   0.988860  -1.450630  -1.049240  -1.932018   0.904318  ...   
       2023-01-1

In [ ]:
fr.read_factor(['球队硬币','云开雾散'],trade_date=20230106)

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据


,,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,301398.SZ,600038.SH,600459.SH,600663.SH,600729.SH,603029.SH,688141.SH,688147.SH,688172.SH,688498.SH
name,index,,,,,,,,,,,,,,,,,,,,,
球队硬币因子,2023-01-06,-3.45854,-0.674148,0.220243,-0.950252,8.786346,0.500339,-1.828122,0.267091,-0.781898,2.299623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
云开雾散因子,2023-01-06,-0.29115,-0.524840,-1.476704,-3.120565,3.906959,-1.295554,-1.253691,-0.280816,0.352601,2.508948,...,2.564407,-0.250353,0.183853,1.403672,-0.088864,10.332152,-0.121816,0.644811,1.282682,0.950078


In [ ]:
fr.read_factor(['球队硬币','云开雾散','飞蛾扑火'],start_date=pd.Timestamp('20230101'),end_date=20230106,sql_like=True) # 以sql的形式，分别读取球队硬币因子、云开雾散因子、飞蛾扑火因子从2023年1月1日至2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据
正在读取飞蛾扑火因子的数据


,date,code,球队硬币因子,云开雾散因子,飞蛾扑火因子
0,2023-01-03,000001.SZ,-2.358642,-0.981326,-1.211546
1,2023-01-03,000002.SZ,-1.345033,-0.944349,-0.747704
2,2023-01-03,000004.SZ,1.162492,-0.997003,-1.218871
3,2023-01-03,000005.SZ,0.007223,4.198288,-0.089553
4,2023-01-03,000006.SZ,8.697314,3.824620,6.631622
...,...,...,...,...,...
20114,2023-01-06,688498.SH,NaN,0.950078,5.589459
20115,2023-01-04,000153.SZ,NaN,NaN,3.513004
20116,2023-01-05,000153.SZ,NaN,NaN,4.299205
20117,2023-01-06,000153.SZ,NaN,NaN,2.936270


In [ ]:
fr.read_factor(['球队硬币','云开雾散','飞蛾扑火'],trade_date=20230106,sql_like=True) # 以sql的形式，分别读取球队硬币因子、云开雾散因子、飞蛾扑火因子2023年1月13日的因子数据

正在读取球队硬币因子的数据
正在读取云开雾散因子的数据
正在读取飞蛾扑火因子的数据


,date,code,球队硬币因子,云开雾散因子,飞蛾扑火因子
0,2023-01-06,000001.SZ,-3.458540,-0.291150,-0.952147
1,2023-01-06,000002.SZ,-0.674148,-0.524840,-0.743429
2,2023-01-06,000004.SZ,0.220243,-1.476704,-1.224969
3,2023-01-06,000005.SZ,-0.950252,-3.120565,-0.757643
4,2023-01-06,000006.SZ,8.786346,3.906959,6.767050
...,...,...,...,...,...
5029,2023-01-06,688147.SH,NaN,0.644811,NaN
5030,2023-01-06,688172.SH,NaN,1.282682,0.124283
5031,2023-01-06,688498.SH,NaN,0.950078,5.589459
5032,2023-01-06,000153.SZ,NaN,NaN,2.936270


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()